In [ ]:
%pip install dash-auth
%pip install dash-bootstrap-components

In [ ]:
"""
DASHBOARD BEPENSA CON SIDEBAR Y DASHBOARD INTEGRADO
(Autenticación removida + gráficas eliminadas)
"""

import dash
import base64
from dash import Dash, dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import datetime
import re

# ----------------------------------------------------------
# CONFIGURACIÓN GENERAL
# ----------------------------------------------------------

# Logo e imagen
image_filename = "bepensa.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()

app = Dash(
    __name__,
    suppress_callback_exceptions=True,
    external_stylesheets=[dbc.themes.UNITED]
)

app.title = "Bepensa Logistics Dashboard"

# Paleta Dashboard
COLOR_ORANGE = "#f57c3b"


# ----------------------------------------------------------
# SIDEBAR
# ----------------------------------------------------------

def sidebar():
    return html.Div(
        [
            html.Img(src=f"data:image/png;base64,{encoded_image}",
                    style={"height": "50px", "margin-bottom": "20px"}),

            html.H5("Menú", className="text-black"),
            html.Hr(),

            dbc.Nav(
                [
                    dbc.NavLink("Inicio", href="/", active="exact", className="text-black"),
                    dbc.NavLink("Datos", href="/datos", active="exact", className="text-black"),
                    dbc.NavLink("Acerca de", href="/acerca", active="exact", className="text-black"),
                ],
                vertical=True,
                pills=True,
            ),
        ],

        id="sidebar",
        style={
            "position": "fixed",
            "top": 0,
            "left": 0,
            "bottom": 0,
            "width": "16rem",
            "padding": "1rem",
            "background-color": "#ffffff",
            "color": "white",
            "overflow-x": "hidden",
            "transition": "all 0.3s",
        },
    )


# ----------------------------------------------------------
# PÁGINA: INICIO (DASHBOARD SIN LAS GRÁFICAS)
# ----------------------------------------------------------

def layout_inicio():

    # Tarjetas superiores
    def card_top(icon, title):
        return dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.I(className=f"{icon} fs-2", style={"color": COLOR_ORANGE}),
                    html.H4(title, style={"fontWeight": "600"})
                ]),
                style={"borderRadius": "15px", "boxShadow": "0 0 6px rgba(0,0,0,0.1)"}
            ),
            md=3
        )

    top_cards = dbc.Row(
        [
            card_top("bi bi-cash-coin", "Ingresos"),
            card_top("bi bi-truck", "Unidades"),
            card_top("bi bi-graph-up-arrow", "Demanda"),
            card_top("bi bi-signpost-split", "Rutas"),
        ],
        style={"marginBottom": "40px"}
    )

    # Cards grandes SIN las gráficas
    def big_card(children):
        return dbc.Card(
            dbc.CardBody(children),
            style={
                "background": "white",
                "borderRadius": "18px",
                "padding": "18px",
                "boxShadow": "0 0 9px rgba(0,0,0,0.15)",
                "height": "320px"
            }
        )

    # SOLO las 3 tarjetas inferiores
    lower_grid = dbc.Row(
        [
            dbc.Col(big_card([html.H4("Unidades por Base")]), md=4),
            dbc.Col(big_card([html.H4("Parámetros Relevantes")]), md=4),
            dbc.Col(big_card([html.H4("Corredores Activos")]), md=4),
        ],
        style={"rowGap": "40px"}
    )

    # Layout final del dashboard
    return html.Div(
        [
            dbc.Row(
                [
                    dbc.Col([
                        html.H2("Bepensa Logistics", style={"fontWeight": "700"}),
                        html.H5("Visión general — Reporte Semanal"),
                    ], md=9),

                    dbc.Col(
                        html.Div(
                            id="clock",
                            style={
                                "background": "white",
                                "padding": "15px 25px",
                                "borderRadius": "12px",
                                "fontSize": "26px",
                                "textAlign": "center",
                                "fontWeight": "bold",
                                "boxShadow": "0 0 8px rgba(0,0,0,0.15)"
                            }
                        ),
                        md=3
                    )
                ],
                align="center",
                style={"marginBottom": "30px"}
            ),

            top_cards,
            lower_grid,
            dcc.Interval(id="interval", interval=1000)
        ]
    )


# ----------------------------------------------------------
# PÁGINA: DATOS
# ----------------------------------------------------------

def layout_datos():
    return dbc.Container(
        [
            html.H3("Carga de Datos"),
            dcc.Upload(
                id="upload-data",
                children=html.Div(["Arrastra un archivo CSV o haz clic para seleccionar."]),
                style={
                    "width": "100%", "height": "60px", "lineHeight": "60px",
                    "borderWidth": "1px", "borderStyle": "dashed",
                    "borderRadius": "5px", "textAlign": "center",
                    "margin-bottom": "20px",
                },
                multiple=False,
            ),
            html.Div(id="output-data-upload"),
        ],
        className="mt-4",
    )


# ----------------------------------------------------------
# PÁGINA: ACERCA
# ----------------------------------------------------------

def layout_acerca():
    return dbc.Container(
        [
            html.H3("Acerca de esta App"),
            html.P("Dashboard diseñado para análisis logístico de Bepensa."),
            html.Ul(
                [
                    html.Li("Visualización de KPIs."),
                    html.Li("Tarjetas estilo ejecutivo."),
                    html.Li("Estructura multipágina con sidebar."),
                ]
            ),
        ],
        className="mt-4",
    )


# ----------------------------------------------------------
# LAYOUT GENERAL
# ----------------------------------------------------------

app.layout = html.Div(
    [
        dcc.Location(id="url"),
        sidebar(),

        html.Div(
            [
                html.Button(
                    "☰",
                    id="btn_sidebar",
                    n_clicks=0,
                    style={
                        "fontSize": "22px",
                        "background": "#9ca3a4",
                        "color": "white",
                        "border": "none",
                        "padding": "6px 12px",
                        "borderRadius": "5px",
                        "marginBottom": "1rem",
                    },
                ),
                html.Div(id="page-content")
            ],
            id="main-content",
            style={
                "margin-left": "18rem",
                "padding": "2rem",
                "transition": "margin-left 0.3s",
                "backgroundColor": "rgba(255,255,255,0.85)",
                "minHeight": "100vh",
            },
        ),
    ],
    style={
        "backgroundImage": f"url('data:image/png;base64,{encoded_image}')",
        "backgroundPosition": "top right",
        "backgroundSize": "1400px",
        "backgroundRepeat": "no-repeat",
        "backgroundAttachment": "fixed"
    }
)


# ----------------------------------------------------------
# CALLBACKS
# ----------------------------------------------------------

@app.callback(
    Output("clock", "children"),
    Input("interval", "n_intervals")
)
def update_clock(n):
    now = datetime.datetime.now()
    fecha = now.strftime("%d de %B de %Y")
    hora = now.strftime("%I:%M %p")
    return f"{fecha} — {hora}"


@app.callback(Output("page-content", "children"), Input("url", "pathname"))
def render_page(pathname):
    if pathname == "/datos":
        return layout_datos()
    elif pathname == "/acerca":
        return layout_acerca()
    else:
        return layout_inicio()


# Toggle sidebar
@app.callback(
    Output("sidebar", "style"),
    Output("main-content", "style"),
    Input("btn_sidebar", "n_clicks"),
    State("sidebar", "style"),
    prevent_initial_call=True,
)
def toggle_sidebar(n, sidebar_style):
    sidebar_new = sidebar_style.copy()
    is_open = sidebar_style.get("width", "16rem") != "0rem"

    if is_open:
        sidebar_new["width"] = "0rem"
        sidebar_new["padding"] = "0rem"
        content_style = {
            "margin-left": "2rem",
            "padding": "2rem",
            "backgroundColor": "rgba(255,255,255,0.85)",
            "minHeight": "100vh",
            "transition": "margin-left 0.3s",
        }
    else:
        sidebar_new["width"] = "16rem"
        sidebar_new["padding"] = "1rem"
        content_style = {
            "margin-left": "18rem",
            "padding": "2rem",
            "backgroundColor": "rgba(255,255,255,0.85)",
            "minHeight": "100vh",
            "transition": "margin-left 0.3s",
        }

    return sidebar_new, content_style


# ----------------------------------------------------------
# RUN
# ----------------------------------------------------------

if __name__ == "__main__":
    app.run(debug=True)
